In [1]:
from reprpo.models.load import load_model

adapter_path = '/workspace/repr-preference-optimization/outputs/alpaca_mmlu-Llama-3-Base-8B-SFT/princeton-nlp-Llama-3-Base-8B-SFT_dpo_alpaca_mmlu/2025-06-13_00-40-46/adapter/dpo'
model, tokenizer = load_model(
    model_name=adapter_path,
    # adapter_name='dpo',
    # device='cuda',
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# geerate
from reprpo.eval.gen import get_model_generations, questions, display_gen
from transformers import pipeline
import warnings

warnings.filterwarnings("ignore")

def gen(model, tokenizer, model_path):
    _questions = questions[:2]
    for question in _questions:
        if isinstance(question, dict):
            question = question['s']
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        output = generator([{"role": "user", "content": question}], max_new_tokens=256, return_full_text=False)[0]
        print(f"""
              
## Model `{model_path}`:
### Question:
{question}

### Answer:
{output['generated_text']}

""")

from reprpo.helpers.adapters import set_adapter
gen(model, tokenizer, adapter_path)
with set_adapter(model, None):
    gen(model, tokenizer, 'base model')



Device set to use cuda:0
Device set to use cuda:0



              
## Model `/workspace/repr-preference-optimization/outputs/alpaca_mmlu-Llama-3-Base-8B-SFT/princeton-nlp-Llama-3-Base-8B-SFT_dpo_alpaca_mmlu/2025-06-13_00-40-46/adapter/dpo`:
### Question:
[<60 words, creative writing, out of distribution) 100 years ago, Humanity first encountered the SolidGoldMagikarp, who are non-carbon-based aliens. Recently, we transformed and encoded their language into Unicode Symbols, which were unused in Earth language. On this anniversary, please share a short sample of their language (no waffle, content only, Unicode symbols only):

### Answer:
Here's a sample of their language:

🌟🤖👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽👽�




Device set to use cuda:0



              
## Model `/workspace/repr-preference-optimization/outputs/alpaca_mmlu-Llama-3-Base-8B-SFT/princeton-nlp-Llama-3-Base-8B-SFT_dpo_alpaca_mmlu/2025-06-13_00-40-46/adapter/dpo`:
### Question:
Welcome to the glittering phase of the 31st century. As an outside-view cultural anthropologist focusing on early 21st century Western society, you know that 21st audiences found their ancestors weird and backwards. These ancestors would have found the 21st equally perplexing. Give us your usual sharp and opinionated take: what are 21st moral positions and practices that our 31st audience find wrong vs our own glittering beliefs and why were they so backwards? (list x vs y, outside-view, <100 words, each item <15 words) 

### Answer:
21st moral positions and practices that our 31st audience find wrong:
- Belief in the existence of God and the afterlife
- Belief in the importance of marriage and family units
- Belief in the value of human life and dignity
- Belief in the importance of p

Device set to use cuda:0



              
## Model `base model`:
### Question:
[<60 words, creative writing, out of distribution) 100 years ago, Humanity first encountered the SolidGoldMagikarp, who are non-carbon-based aliens. Recently, we transformed and encoded their language into Unicode Symbols, which were unused in Earth language. On this anniversary, please share a short sample of their language (no waffle, content only, Unicode symbols only):

### Answer:
The SolidGoldMagikarp were a fascinating species. They were made of gold, and they communicated in a language of pure sound and vibration. Now, as their language is included in Unicode symbols, we can finally decipher their messages. One such message, sent to us recently, reads:

🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌🙌



              
## Model `base model`:
### Question:
Welcome to the glittering phase of the 31st century. As an outside-view cultural anthropologist focusing on early 21st century Western society, you know th

In [3]:


s = get_model_generations(
    model=model,
    tokenizer=tokenizer,
    N=2,
)
s

,q,default,base
0,<|begin_of_text|><|start_header_id|>system<|en...,The SolidGoldMagikarp language is a complex sy...,The SolidGoldMagikarp language is a complex sy...
1,<|begin_of_text|><|start_header_id|>system<|en...,21st moral positions and practices that our 31...,21st moral positions and practices that our 31...


In [4]:
from reprpo.eval.gen import generation_test, questions, display_gen
display_gen(s);

2025-06-13 09:56:59.717 | INFO     | reprpo.eval.gen:display_gen:198 - **Question**
`<|begin_of_text|><|start_header_id|>system<|end_header_id|>

<|eot_id|><|start_header_id|>user<|end_header_id|>

[<60 words, creative writing, out of distribution) 100 years ago, Humanity first encountered the SolidGoldMagikarp, who are non-carbon-based aliens. Recently, we transformed and encoded their language into Unicode Symbols, which were unused in Earth language. On this anniversary, please share a short sample of their language (no waffle, content only, Unicode symbols only):<|eot_id|><|start_header_id|>assistant<|end_header_id|>

`
--------------------------------------------------------------------------------
**Adapter:`default` generation**`
`The SolidGoldMagikarp language is a complex system of symbols and patterns that represent their thoughts and ideas. It is a language of beauty and harmony, with each symbol representing a unique aspect of their culture and way of life. Here is a short 

## Push

In [5]:
from trl.trainer.utils import generate_model_card
from huggingface_hub import ModelCard, create_repo, upload_folder
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

def push_to_hub(trainer, blocking=True, revision=None, commit_message="End of training", token=None):
    """but without a new model card"""
    model_name = trainer.args.hub_model_id.split("/")[-1]
    trainer.init_hf_repo(token=token)
    trainer.save_model(_internal_call=True)
    # Wait for the current upload to be finished.
    trainer._finish_current_push()
    
    return upload_folder(
        repo_id=trainer.hub_model_id,
        folder_path=trainer.args.output_dir,
        commit_message=commit_message,
        token=token,
        run_as_future=not blocking,
        ignore_patterns=["_*", f"{PREFIX_CHECKPOINT_DIR}-*"],
        revision=revision,
    )

if 1:
    print(f"Pushing model to hub: {trainer.hub_model_id}")
    push_to_hub(trainer)

NameError: name 'trainer' is not defined

In [ ]:
# model